In [122]:
from model import *
from vocab import Vocab
from dataset import SSTDataset
from metrics import Metrics
from utils import load_word_vectors, build_vocab
from config import parse_args
from trainer import SentimentTrainer

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# TODO: refactor
import os
vocab_file = 'vocab.txt'
train_dir = 'training-treebank'
vocab = Vocab(filename=vocab_file)
dataset = SSTDataset(train_dir, vocab)
splitting_point = int(len(dataset)*0.9)
train_dataset, dev_dataset = SSTDataset(num_classes=3), SSTDataset(num_classes=3)
train_dataset.trees = dataset.trees[:splitting_point]
train_dataset.sentences = dataset.sentences[:splitting_point]
train_dataset.labels = dataset.labels[:splitting_point]
dev_dataset.trees = dataset.trees[splitting_point:]
dev_dataset.sentences = dataset.sentences[splitting_point:]
dev_dataset.labels = dataset.labels[splitting_point:]

100%|██████████| 965/965 [00:00<00:00, 72404.85it/s]
235it [00:00, 12789.67it/s]
965it [00:00, 7730.69it/s]


In [4]:
# initialize model, criterion/loss_function, optimizer
model = TreeLSTMSentiment(
            False, vocab.size(),
            200, 168,
            3, '', criterion = nn.NLLLoss()
        )
embedding_model = nn.Embedding(vocab.size(), 200)

In [5]:
emb_file = os.path.join('training-treebank', 'sst_embed.pth')
emb = torch.load(emb_file)
embedding_model.state_dict()['weight'].copy_(emb)


-1.9076e-02 -1.6825e-02 -1.4530e-01  ...  -2.2152e-02 -1.0355e-01 -1.1936e-01
-8.3834e-02  1.2338e-03 -4.2120e-02  ...  -3.6188e-02  9.4087e-02 -6.0075e-02
-1.0376e-01  1.0742e-02 -2.2019e-03  ...  -5.9040e-02 -2.9827e-02  4.8783e-02
                ...                   ⋱                   ...                
-9.8814e-02 -3.0921e-02 -1.0864e-01  ...   2.4859e-02 -7.0311e-02 -3.3570e-02
-3.5013e-01 -3.6512e-01  1.7883e-01  ...   6.6884e-01 -1.1198e+00  1.9719e-01
-3.6237e-01 -7.8314e-01 -3.3223e-01  ...  -3.8631e-01 -1.6861e-01 -1.4218e-01
[torch.FloatTensor of size 4114x200]

In [124]:
model = torch.load('saved_model9_model_default_name.pth')
embedding_model = torch.load('saved_model9_embedding_default_name.pth')
trainer = SentimentTrainer(None, model, embedding_model, 
                           criterion=nn.NLLLoss(), optimizer=None)
test_loss, test_pred, trees = trainer.test(dev_dataset)

/Users/tomek/.virtualenvs/x/lib/python3.6/site-packages/torch/serialization.py:284: SourceChangeWarning: source code of class 'model.ChildSumTreeLSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
Testing epoch  0: 100%|██████████| 120/120 [00:01<00:00, 90.94it/s]


In [125]:
for tree in trees:
    print(torch.exp(tree.output))
    print(tree._viz_sentence)
    print(tree.gold_label)

Variable containing:
 0.1774  0.3954  0.4272
[torch.FloatTensor of size 1x3]

['Są', 'to', 'trwałe', 'perfumy', 'które', 'nie', 'kosztują', 'dużo', '.']
1
Variable containing:
 0.0305  0.9602  0.0093
[torch.FloatTensor of size 1x3]

['Kolejną', 'aferę', 'w', 'zakładach', 'pracy', 'chronionej', 'ujawnili', 'policjanci', 'z', 'wydziału', 'do', 'walki', 'z', 'przestępczością', 'gospodarczą', 'Komendy', 'Wojewódzkiej', 'w', 'Olsztynie', '.']
1
Variable containing:
 0.0275  0.9479  0.0246
[torch.FloatTensor of size 1x3]

['Jeśli', 'chodzi', 'o', 'cenę', 'to', 'jest', 'ona', 'warta', 'tej', 'jakości', '.']
1
Variable containing:
 0.0494  0.6414  0.3091
[torch.FloatTensor of size 1x3]

['Dolce', '&', 'Gabbana', 'stworzony', 'dla', 'kobiety', 'nowoczesnej', ',', 'bardzo', 'kobiecej', '.']
1
Variable containing:
 0.0353  0.0081  0.9567
[torch.FloatTensor of size 1x3]

['Ma', 'odpinane', 'ramiączka', 'i', 'słodką', 'kokardkę', 'między', 'piersiami', '.']
1
Variable containing:
 0.0071  0.9839  0

In [130]:
from spacy import displacy
for index, tree in enumerate(trees):
    tree.visualize()
    try:
        displacy.render(
            tree.visualize(with_output=True), 
            style='dep', manual=True, jupyter=True, options={'compact': False})
    except:
        print('something goes wrong sometimes, dont know why, nvm')
    if index > 50:
        break

no ej
[<tree.Tree object at 0x102779f98>, <tree.Tree object at 0x10277d0b8>, <tree.Tree object at 0x10277d208>]
Dolce
2
something goes wrong sometimes, dont know why, nvm
